# use astropy to anaylize fits images

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [ ]:
fname = "HorseHead.fits"
hdu_list = fits.open(fname)
hdu_list.info()

# Generally the image info is located in the PRIMARY block, the blocks are numbered and can be accessed by indexing hdu_list

In [ ]:
image_data = hdu_list[0].data

# Our data is now stored as a 2d numpy array. But how do we know the dimensions of te image? we can look at the sape of the array.


In [ ]:
print(type(image_data))
print(image_data.shape)

# At this pt, we can close the FITS file because we've stored everything we wanted to a variable.


In [ ]:
hdu_list.close()

# Shortcut: use "getdata()" to just read in the image data and close the file

In [ ]:
image_data = fits.getdata(fname)
print(type(image_data))
print(image_data.shape)

In [ ]:
plt.imshow(image_data, cmap='magma')
plt.colorbar()

# Lets get some basic stats about our image

In [ ]:
print('Min:', np.min(image_data))
print("Max:", np.max(image_data))
print('Mean:', np.mean(image_data))
print('Stdev:', np.std(image_data))

# Plotting a histogram

In [ ]:
#To make a histogram w/ matplotlib.pyplot.hist(), we'll need to cast the data from a 2D array to something one 
#dimensional
#in this case, let's use the ndarray.flatten() to return a 1D numpy array

histogram = plt.hist(image_data.flatten(), bins='auto')

# Displaying the image w/ a log scale

In [ ]:
#what if we want to use a log color scale? Load the LogNorm object from matplotlib.

from matplotlib.colors import LogNorm

In [ ]:
plt.imshow(image_data, cmap='magma', norm=LogNorm())

#Choose the tick marks based on the histogram above
cbar = plt.colorbar(ticks=[5.e3, 1.e4, 2.e4])
cbar.ax.set_yticklabels(['5,000', '10,000', '20,000'])

# Stacking Images

In [ ]:
#Since the noise in an image results from a random process,we use stacking of separate images to improve the signal to
#noise ratio of objects we observe. Here we are going to stack 5 images of M13 taken with a 10 inch telescope.

In [ ]:
#make a list of filenames
image_list = ['M13_blue_0001.fits', 'M13_blue_0002.fits', 'M13_blue_0003.fits', \
              'M13_blue_0004.fits', 'M13_blue_0005.fits']

In [ ]:
#make an array of images from tje list of images
image_concat = [fits.getdata(image) for image in image_list]

In [ ]:
#sum the images together
final_image = np.sum(image_concat, axis=0)

In [ ]:
#plot a histogram of the image pixel values
image_hist = plt.hist(final_image.flatten(), bins='auto')

In [ ]:
#We'll use the keyword vmin and vmax to set limits on the color scaling for imshow.

In [ ]:
plt.imshow(final_image, cmap='gray', vmin=2.5E3, vmax=3E3)
plt.colorbar()

# Writing a new FITS file

In [ ]:
#We can easily do this with the writeto() method.
#WARNING you'll recieve an error if the file you are trying to write already exists. That's why we've set
#clobber=True.

In [ ]:
outfile = 'stacked_M13_blue.fits'
hdu = fits.PrimaryHDU(final_image)
hdu.writeto(outfile, overwrite=True)